In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import os
import shutil
import kagglehub
organizations_bigquery_bitcoin_blockchain_path = kagglehub.dataset_download('organizations/bigquery/bitcoin-blockchain')

print('Data source import complete.')


In [ ]:
from google.cloud import bigquery
from bq_helper import BigQueryHelper


In [ ]:
bq_assistant = BigQueryHelper("bigquery-public-data", "bitcoin_blockchain")



Using Kaggle's public dataset BigQuery integration.


# Queries

In [ ]:
q1 = '''
    SELECT AVG(size) AS Average_Size, AVG(stripped_size) AS Average_stripped_Size, AVG(weight) AS Average_weight FROM `bigquery-public-data.crypto_bitcoin.blocks`
    '''
client = bigquery.Client()
query_job = client.query(q1)
q1 = query_job.to_dataframe()
q1

Using Kaggle's public dataset BigQuery integration.


,Average_Size,Average_stripped_Size,Average_weight
0,702616.365243,485246.410972,2.158356e+06


In [ ]:
q2 = '''
    SELECT type, COUNT(type) AS Frequency FROM `bigquery-public-data.crypto_bitcoin.outputs`GROUP BY type ORDER BY COUNT(type) DESC
    '''
query_job = client.query(q2)
q2 = query_job.to_dataframe()
q2

,type,Frequency
0,pubkeyhash,1361687863
1,scripthash,737410503
2,witness_v0_keyhash,573891896
3,witness_v1_taproot,218569467
4,nonstandard,124956391
5,witness_v0_scripthash,39893739
6,pubkey,2997650
7,multisig,677099
8,witness_unknown,125


In [ ]:
trans_day = '''
            SELECT
              DATE(block_timestamp) AS date,
              COUNT(*) AS no_transactions
              FROM `bigquery-public-data.crypto_bitcoin.transactions`
              WHERE DATE(block_timestamp) <= DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY) AND is_coinbase IS FALSE
              GROUP BY date
              ORDER BY date
            '''
query_job = client.query(trans_day)
trans_day = query_job.to_dataframe()
trans_day

,date,no_transactions
0,2009-01-12,7
1,2009-01-14,1
2,2009-01-15,8
3,2009-01-16,2
4,2009-01-18,1
...,...,...
5501,2024-10-12,736178
5502,2024-10-13,648601
5503,2024-10-14,611544
5504,2024-10-15,646735


In [ ]:
q3 = """ WITH time AS
            (
                SELECT TIMESTAMP_MILLIS(timestamp) AS trans_time,
                    transaction_id
                FROM `bigquery-public-data.bitcoin_blockchain.transactions`
            )
            SELECT COUNT(transaction_id) AS transactions,
                EXTRACT(MONTH FROM trans_time) AS month,
                EXTRACT(YEAR FROM trans_time) AS year
            FROM time
            GROUP BY year, month
            ORDER BY year, month
        """
query_job = client.query(q3)
q3 = query_job.to_dataframe()
q3


,transactions,month,year
0,2575,1,2009
1,3417,2,2009
2,3487,3,2009
3,3459,4,2009
4,3401,5,2009
...,...,...,...
112,6138957,5,2018
113,5752514,6,2018
114,6393536,7,2018
115,6417764,8,2018


In [ ]:
q4='''
    SELECT
        DATE(block_timestamp) AS date,
        AVG(input_count) AS avg_input,
        AVG(output_count) AS avg_output,
        MAX(input_count) AS max_input,
        MAX(output_count) AS max_output,
        SAFE_DIVIDE(SUM(output_count), SUM(input_count)) AS ratio
FROM `bigquery-public-data.crypto_bitcoin.transactions`
WHERE input_count != 0  AND input_value != 0 AND output_value != 0
GROUP BY date
ORDER BY date
    '''
query_job = client.query(q4)
q4 = query_job.to_dataframe()
q4

,date,avg_input,avg_output,max_input,max_output,ratio
0,2009-01-12,1.000000,1.714286,1,2,1.714286
1,2009-01-14,3.000000,1.000000,3,1,0.333333
2,2009-01-15,1.875000,1.750000,5,2,0.933333
3,2009-01-16,2.000000,1.000000,2,1,0.500000
4,2009-01-18,3.000000,1.000000,3,1,0.333333
...,...,...,...,...,...,...
5502,2024-10-13,1.656824,2.462086,1375,2009,1.486028
5503,2024-10-14,1.816195,2.470300,1469,1995,1.360151
5504,2024-10-15,1.925067,2.461392,1441,1978,1.278601
5505,2024-10-16,1.533396,2.435772,1197,2021,1.588482


In [ ]:
q5 = '''
    SELECT
    DATE(block_timestamp) as transaction_date,
    SUM(output.value) as total_bitcoins_sent
FROM
    `bigquery-public-data.crypto_bitcoin.transactions`
JOIN
    UNNEST(outputs) AS output
GROUP BY
    transaction_date
ORDER BY
    transaction_date DESC
LIMIT
    30;
    '''
query_job = client.query(q5)
q5 = query_job.to_dataframe()
q5

,transaction_date,total_bitcoins_sent
0,2024-10-17,9788085682679.000000000
1,2024-10-16,70187496734992.000000000
2,2024-10-15,83634190135389.000000000
3,2024-10-14,60596304137674.000000000
4,2024-10-13,36231035458436.000000000
5,2024-10-12,38699374413750.000000000
6,2024-10-11,63970605370957.000000000
7,2024-10-10,64215151962659.000000000
8,2024-10-09,81227019035577.000000000
9,2024-10-08,106866800704866.000000000
